## 가게명 코사인 유사도 

**목표**<br>
챗봇에서 가게명을 입력 받을 때 데이터셋에 있는 가게명과 정확하게 일치하지 않아도 코사인 유사도 모델을 통해 비슷한 가게명을 선택할 수 있다.

- 가게리스트 데이터의 storename 컬럼을 이용해 가게명 데이터셋을 만든다.
- 위 데이터셋을 이용해 각 가게명별 코사인 유사도를 나타내는 모델을 만든다.


#### 코사인 유사도 설명 참고자료
- https://wikidocs.net/24603

In [1]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Mecab

### 1. 음식메뉴 데이터 불러오기 

In [2]:
# 가게리스트 데이터 불러오기
df_restaurants = pd.read_csv('./questions/가게리스트_카테고리_최종.csv',encoding='utf-8',index_col=0)
df_restaurants.head(2)

,storeid,Title,storename,Region,Sigungu,search_area,Category1,Category2,menu,Blog,reviews_num
0,13992,빛고을떡갈비,빛고을떡갈비,광주,광산구,광산,한식,숯불구이,육회비빔밥,24,2
1,13995,착한소장수,착한소장수,광주,동구,광주,한식,갈비,갈비살/안창살/살치살/왕갈비탕,4155,7


In [13]:
# storename 컬럼 저장
df_storenames = df_restaurants['storename'].to_frame()
df_storenames.shape

(22782, 1)

In [14]:
df_storenames.head(10)

,storename
0,빛고을떡갈비
1,착한소장수
2,고흥나루터
3,고흥장어나라
4,관가
5,미연
6,백운산
7,원조장수
8,제일중화요리
9,청해복집


### 2. `TfidfVectorizer`를 이용하여 벡터화

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
# 공백 제거하는 함수
def text_prepare(text):
    return text.replace(' ','')

In [17]:
df_storenames['storename_prep'] = df_storenames['storename'].apply(text_prepare)
df_storenames.head(5)

,storename,storename_prep
0,빛고을떡갈비,빛고을떡갈비
1,착한소장수,착한소장수
2,고흥나루터,고흥나루터
3,고흥장어나라,고흥장어나라
4,관가,관가


가게명 중에서 가장 긴 음절을 가진 가게명을 찾는다.

In [21]:
storename_len = df_storenames.storename_prep.apply(lambda x: len(x)).sort_values(ascending=False)
storename_len

2025     30
9731     30
16452    28
4087     26
1149     25
         ..
770       1
3184      1
3309      1
9137      1
686       1
Name: storename_prep, Length: 22782, dtype: int64

In [40]:
# 테스트를 위해 긴 음절 상위 20개, 짧은 음절 하위 20개를 저장한다.
long_storenames = storename_len[:20]
short_storenames = storename_len[-20:]

In [22]:
storename_len.describe()

count    22782.000000
mean         4.878501
std          2.191600
min          1.000000
25%          4.000000
50%          5.000000
75%          6.000000
max         30.000000
Name: storename_prep, dtype: float64

In [20]:
# 가장 긴 음절은 30음절
df_storenames.iloc[2025].storename

'고기듬뿍대왕비빔밥&커피몬스터(COFFEEMONSTER)'

가게명 음절 개수의 평균값은 5이므로, ngram_range를 5로 선택한다.

In [45]:
# TF-IDF vectorizer 생성
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,5) # 1음절~4음절까지 검색
                                   ,analyzer='char')  # 음절 단위 분석

# 벡터화할 데이터를 저장
y = df_storenames.storename_prep.values

# 벡터화된 데이터의 지역명을 저장
y_storename = df_storenames.storename.values

# vectorizer를 y 데이터로 fitting
tfidf_vectorizer.fit(y)

# fitting 된 vectorizer 확인
tfidf_vectorizer

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 5), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [47]:
# tfidf에 사용된 형태소 단어 확인 : 22,782개 가게명을 나타내기 위해 총 127,914 음절이 사용되었음
tfidf_vocabs = tfidf_vectorizer.vocabulary_
len(tfidf_vocabs)

127914

In [48]:
# 음절 미리보기
tfidf_vocabs

{'빛': 61038,
 '고': 10279,
 '을': 91040,
 '떡': 35335,
 '갈': 7384,
 '비': 60087,
 '빛고': 61039,
 '고을': 11097,
 '을떡': 91122,
 '떡갈': 35342,
 '갈비': 7458,
 '빛고을': 61040,
 '고을떡': 11105,
 '을떡갈': 91123,
 '떡갈비': 35343,
 '빛고을떡': 61043,
 '고을떡갈': 11106,
 '을떡갈비': 91124,
 '빛고을떡갈': 61044,
 '고을떡갈비': 11107,
 '착': 105997,
 '한': 121360,
 '소': 68233,
 '장': 96720,
 '수': 70380,
 '착한': 106000,
 '한소': 122087,
 '소장': 68842,
 '장수': 97341,
 '착한소': 106092,
 '한소장': 122094,
 '소장수': 68843,
 '착한소장': 106095,
 '한소장수': 122095,
 '착한소장수': 106096,
 '흥': 127622,
 '나': 19609,
 '루': 39444,
 '터': 114343,
 '고흥': 11579,
 '흥나': 127652,
 '나루': 19888,
 '루터': 39648,
 '고흥나': 11580,
 '흥나루': 127653,
 '나루터': 19898,
 '고흥나루': 11581,
 '흥나루터': 127654,
 '고흥나루터': 11582,
 '어': 80787,
 '라': 36028,
 '흥장': 127742,
 '장어': 97575,
 '어나': 80894,
 '나라': 19811,
 '고흥장': 11594,
 '흥장어': 127743,
 '장어나': 97590,
 '어나라': 80898,
 '고흥장어': 11595,
 '흥장어나': 127744,
 '장어나라': 97591,
 '고흥장어나': 11596,
 '흥장어나라': 127745,
 '관': 12987,
 '가': 5637,
 '관가': 12988,
 '미': 50099,
 

In [49]:
# y 데이터 벡터화
y_vectorized = tfidf_vectorizer.transform(y)

# 벡터화된 희소행렬 확인 
# (22,782*126,789 행렬, 22,782개 음식메뉴가 총 126,789개 음절 벡터를 이용하여 벡터화되었음)
# (226,182개 element가 있으므로, 한 행당 평균 10개 정도의 element가 있음)
y_vectorized

<22782x127914 sparse matrix of type '<class 'numpy.float64'>'
	with 334861 stored elements in Compressed Sparse Row format>

### 3. cosine 유사도 모델 생성 : 데이터 내부
- "y데이터 내부"에서 서로 비슷한 음식메뉴는 무엇인지 찾는 모델 생성
- [linear_kernel](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.linear_kernel.html) 이용

In [50]:
from sklearn.metrics.pairwise import linear_kernel

# y_vectorized 데이터를 이용해 linear_kernel 행렬 생성
cosine_sim = linear_kernel(y_vectorized, y_vectorized)

In [51]:
cosine_sim.shape

(22782, 22782)

In [52]:
def get_similar_indices(idx, cosine_sim=cosine_sim):
    """
        idx를 "y데이터 내부"에서 서로 비슷한 가게명는 무엇인지 찾는 모델 생성
    """

    # 모든 가게명에 대해서 해당 idx와의 유사도를 구한다. ((index, linear_kernel값)의 list 형태)
    sim_scores = list(enumerate(cosine_sim[idx])) 

    # 유사도에 따라 가게명을 정렬한다. (key 파라미터는 정렬기준을 선택)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 유사도가 0 이상인 것만 sim_list에 저장한다.
    sim_list=[]
    for i in range(len(sim_scores)):
        if sim_scores[i][1] > 0:
            sim_list.append(sim_scores[i])
            
    # sim_list에서 유사도가 가장 높은 1개만 저장한다.
    sim_list = sim_list[:1]
            
    # sim_list의 인덱스를 저장한다.
    sim_indices = [i[0] for i in sim_list]

    # 5개의 가게명을 list 타입으로 반환한다.
    return sim_indices

긴 음절의 가게명을 정확하게 예측하는지 살펴본다.

In [53]:
test_idxs = long_storenames.index
for idx in test_idxs:
    test_storename = y_storename[idx]
    sim_indices = get_similar_indices(idx)
    sim_storename = [y_storename[idx] for idx in sim_indices]
    print("'",test_storename,"'",'가장 유사한 가게명 1개:', sim_storename)

' 고기듬뿍대왕비빔밥&커피몬스터(COFFEEMONSTER) ' 가장 유사한 가게명 1개: ['고기듬뿍대왕비빔밥&커피몬스터(COFFEEMONSTER)']
' 카페앤레스토랑브뤼(Cafe&RestaurantBRUT) ' 가장 유사한 가게명 1개: ['카페앤레스토랑브뤼(Cafe&RestaurantBRUT)']
' 미스터보쌈5379&STANDINGSTEAK동인천역점 ' 가장 유사한 가게명 1개: ['미스터보쌈5379&STANDINGSTEAK동인천역점']
' 낭만치킨포차동구과학대점엄니국물닭발&탕탕탕(본점) ' 가장 유사한 가게명 1개: ['낭만치킨포차동구과학대점엄니국물닭발&탕탕탕(본점)']
' 307버거앤그릴(307Burger&Grill) ' 가장 유사한 가게명 1개: ['307버거앤그릴(307Burger&Grill)']
' 위쉐프앤바리스타(We'chef&barista) ' 가장 유사한 가게명 1개: ["위쉐프앤바리스타(We'chef&barista)"]
' 한국외식프랜차이즈협동조합(참맛있게&델리커시) ' 가장 유사한 가게명 1개: ['한국외식프랜차이즈협동조합(참맛있게&델리커시)']
' 나는조선의떡볶이다&내가조선의닭발이다거제고현점 ' 가장 유사한 가게명 1개: ['나는조선의떡볶이다&내가조선의닭발이다거제고현점']
' 원스인어블루문(OnceInABluemoon) ' 가장 유사한 가게명 1개: ['원스인어블루문(OnceInABluemoon)']
' 큰가마솥할매순대국&양선지해장국증포점(고풍) ' 가장 유사한 가게명 1개: ['큰가마솥할매순대국&양선지해장국증포점(고풍)']
' 원할머니보쌈,족발&박가부대찌개(하남미사점) ' 가장 유사한 가게명 1개: ['원할머니보쌈,족발&박가부대찌개(하남미사점)']
' 텍사스데브라질(TexasdeBrazil) ' 가장 유사한 가게명 1개: ['텍사스데브라질(TexasdeBrazil)']
' 마켓로커스이마트수지점'My고기&제면명가' ' 가장 유사한 가게명 1개: ["마켓로커스이마트수지점'My고기&제면명가'"]
' 불타는장작구이&치맥먹고노가리풀자(고잔점

20음절이 넘는 가게명도 2~5그램의 음절 단위의 tfidf_vectorizer로 정확하게 예측했다.

1-2음절의 짧은 가게명도 예측하는지 살펴본다

In [54]:
test_idxs = storename_len[-200:].index
for idx in test_idxs:
    test_storename = y_storename[idx]
    sim_indices = get_similar_indices(idx)
    sim_storename = [y_storename[idx] for idx in sim_indices]
    print("'",test_storename,"'",'가장 유사한 가게명 1개:', sim_storename)

' 몽촌 ' 가장 유사한 가게명 1개: ['몽촌']
' 양군 ' 가장 유사한 가게명 1개: ['양군']
' 다함 ' 가장 유사한 가게명 1개: ['다함']
' 대판 ' 가장 유사한 가게명 1개: ['대판']
' 미니 ' 가장 유사한 가게명 1개: ['미니']
' 민하 ' 가장 유사한 가게명 1개: ['민하']
' 마녀 ' 가장 유사한 가게명 1개: ['마녀']
' 윤정 ' 가장 유사한 가게명 1개: ['윤정']
' 이지 ' 가장 유사한 가게명 1개: ['이지']
' 방자 ' 가장 유사한 가게명 1개: ['방자']
' 덴버 ' 가장 유사한 가게명 1개: ['덴버']
' 범양 ' 가장 유사한 가게명 1개: ['범양']
' 보말 ' 가장 유사한 가게명 1개: ['보말']
' 누나 ' 가장 유사한 가게명 1개: ['누나']
' 식당 ' 가장 유사한 가게명 1개: ['식당']
' 보그 ' 가장 유사한 가게명 1개: ['보그']
' 명주 ' 가장 유사한 가게명 1개: ['명주']
' 보트 ' 가장 유사한 가게명 1개: ['보트']
' 썰매 ' 가장 유사한 가게명 1개: ['썰매']
' 부현 ' 가장 유사한 가게명 1개: ['부현']
' 보경 ' 가장 유사한 가게명 1개: ['보경']
' 혜성 ' 가장 유사한 가게명 1개: ['혜성']
' 로이 ' 가장 유사한 가게명 1개: ['로이']
' 보금 ' 가장 유사한 가게명 1개: ['보금']
' 다울 ' 가장 유사한 가게명 1개: ['다울']
' 별당 ' 가장 유사한 가게명 1개: ['별당']
' 비어 ' 가장 유사한 가게명 1개: ['비어']
' 냉정 ' 가장 유사한 가게명 1개: ['냉정']
' 시오 ' 가장 유사한 가게명 1개: ['시오']
' 어군 ' 가장 유사한 가게명 1개: ['어군']
' 목마 ' 가장 유사한 가게명 1개: ['목마']
' 복지 ' 가장 유사한 가게명 1개: ['복지']
' 낯선 ' 가장 유사한 가게명 1개: ['낯선']
' 복집 ' 가장 유사한 가게명 1개: ['복집']
' 믿고 ' 가장 유사한 

전체 가게명에 대해 정확도를 알아본다.

In [66]:
accurate = 0
for idx in storename_len.index:
    test_storename = y_storename[idx]
    sim_idx = get_similar_indices(idx)[0]
    sim_storename = y_storename[sim_idx]
    if test_storename == sim_storename:
        accurate += 1
accuracy = (accurate / len(storename_len)) * 100
print('전체 가게명 정확도:',str(accuracy), '%')

전체 가게명 정확도: 99.99561056974805 %


### 4. cosine 유사도 모델 생성 : 데이터 외부
- df_storenames 데이터와 살짝 다른 가게명이 입력되었을 때, df_storenames 내에서 가장 비슷한 가게명은 무엇인지 찾는 모델 생성
- [cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) 이용

#### 가게명과 조사를 섞어서 테스트

In [67]:
from sklearn.metrics.pairwise import cosine_similarity

In [68]:
def get_similar_storename(new_storename, tfidf_vectorizer, y_vectorized):
    """
        데이터에 없는 새로운 지역명 new_storename을 입력 받고, 
        new_storename과 가장 유사한 지역명 3개와 각각의 유사도를 데이터프레임 형태로 반환
    """
    
    # new_storename 형태소 분석
    new_storename_mecab = [text_prepare(new_storename)]
    
    # new_storename 벡터화
    new_storename_vectorized = tfidf_vectorizer.transform(new_storename_mecab)
    
    # 유사한 메뉴와 유사도를 저장
    sim_storename = {}

    for i in range(y_vectorized.shape[0]):
        curr_sim = cosine_similarity(y_vectorized[i], new_storename_vectorized)[0,0]
        if curr_sim > 0.2:
            sim_storename[y_storename[i]] = curr_sim

    sim_storename = pd.DataFrame(data=sim_storename.values()
                           ,index=sim_storename.keys()
                           ,columns=['cs']).sort_values('cs',ascending=False)

    return sim_storename[:3]

In [74]:
cosine_similarity

<22782x127914 sparse matrix of type '<class 'numpy.float64'>'
	with 334861 stored elements in Compressed Sparse Row format>

In [73]:
new_storename_mecab = [text_prepare(new_storename)]
    
# new_storename 벡터화
new_storename_vectorized = tfidf_vectorizer.transform(new_storename_mecab)

cosine_similarity(y_vectorized[1], new_storename_vectorized)[0,0]

NameError: name 'new_storename_vectorized' is not defined

In [70]:
test_storenames = [
    '버거앤그릴', '놀부 부대찌개에', '서가앤 쿡으로', '맛있는 장충동왕족발보쌈'
]

print('가장 유사한 가게명 3가지')
for storename in test_storenames:
    print(storename,'\t:',get_similar_storename(storename,tfidf_vectorizer,y_vectorized).index.tolist())
    print('\tscore:',get_similar_storename(storename,tfidf_vectorizer,y_vectorized)['cs'].tolist())

가장 유사한 가게명 3가지
버거앤그릴 	: ['포앤그릴', '그릴앤그릴', '그릴']
	score: [0.40793279971074026, 0.4049057566180899, 0.3751543018576156]
놀부 부대찌개에 	: ['놀부부대찌개', '부부부대찌개', '놀부보쌈놀부부대찌개']
	score: [0.981800260133794, 0.6632435297617345, 0.6414649645007604]
서가앤 쿡으로 	: ['서가앤쿡목동점', '서가앤쿡부평점', '쿡']
	score: [0.5382938070492439, 0.5374590376913125, 0.2848228755793984]
맛있는 장충동왕족발보쌈 	: ['소문난장충동왕족발보쌈', '장충동왕족발보쌈(동탄점)', '장충동왕족발보쌈&에꿍이치킨']
	score: [0.7110058051915322, 0.6257671025210176, 0.5852987493215978]


**pickle 저장**

In [71]:
import pickle

with open('./cosine_storename_tfidf_vectorizer.pickle', "wb") as f:
    pickle.dump(tfidf_vectorizer, f, pickle.HIGHEST_PROTOCOL)
with open('./cosine_storename_y_storename.pickle', "wb") as f:
    pickle.dump(y_storename, f, pickle.HIGHEST_PROTOCOL)
with open('./cosine_storename_y_vectorized.pickle', "wb") as f:
    pickle.dump(y_vectorized, f, pickle.HIGHEST_PROTOCOL)
with open('./cosine_storename_cosine_sim.pickle', "wb") as f:
    pickle.dump(cosine_sim, f, pickle.HIGHEST_PROTOCOL)

**클래스 정리**